<img src = "https://drive.google.com/uc?export=view&id=1G392UZpxYkEBtf7em1zTV7tU3jMv49i0" alt = "Encabezado MLDS" width = "100%">  </img>

# **_MongoDB_ con _Python_**
---
En este notebook se dará una breve introducción práctica a _PyMongo_, el _driver_ oficial para el lenguaje de programación _Python_ de la base de datos NoSQL basada en documentos [_MongoDB_](https://www.mongodb.com/). Para esto, se retomarán los mismos comandos ejecutados en el taller guiado de operaciones en consola de la herramienta, con el objetivo de comparar los dos métodos.

## **0. Instalar _MongoDB_ en _Google Colab_**
---

_PyMongo_ es una herramienta que permite conectar un entorno de ejecución de _Python_ a un servidor activo de _MongoDB_. No es parte de su funcionalidad ejecutar este tipo de servidor, por lo que dependemos de una instalación local o remota del servicio.


> En este caso, y para facilitar las cosas, vamos a instalar y ejecutar _Mongo_ en la máquina _Ubuntu_ que ofrece _Google Colab_. Para realizar la conexión con un servidor propio deberá habilitar la configuración de _MongoDB_ para conexiones remotas y abrir el puerto necesario (comúnmente el puerto **27017**) de la máquina remota. Realizar este tipo de configuración está por fuera del alcance de esta guía. Para más información lo invitamos a profundizar en foros oficiales y guías como la [siguiente](https://www.digitalocean.com/community/tutorials/how-to-configure-remote-access-for-mongodb-on-ubuntu-20-04-es).

Vamos a instalar _MongoDB_ en la máquina del entorno de ejecución con los comandos **`apt update`** y **`apt install`**:

In [1]:
# Actualizamos el repositorio e instalamos mongodb.
!sudo apt -qq update
!sudo apt -qq install -y mongodb

39 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following additional packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb-clients
  mongodb-server mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb mongodb-clients
  mongodb-server mongodb-server-core
0 upgraded, 8 newly installed, 0 to remove and 39 not upgraded.
Need to get 53.1 MB of archives.
After this operation, 215 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 8.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open s

Creamos las carpetas **`data`** y  **`log`** y ejecutamos el servicio **`mongod`**, tal como se realizó en la guía de operaciones en consola:

In [2]:
# Creamos los directorios 'data' y 'log'.
!mkdir -p data log

# Ejecutamos el servicio de Mongo.
!mongod --dbpath ./data --logpath ./log/mongod.log --fork

about to fork child process, waiting until server is ready for connections.
forked process: 1324
child process started successfully, parent exiting


Si todo sale bien, el servicio de _Mongo_ estará activo y expuesto en el puerto **27017** de la máquina local.

## **1. Instalar dependencias**
---

Antes de comenzar con la herramienta, vamos a instalar los módulos de _Python_ utilizados en esta guía, empezando por **`pymongo`**:

In [3]:
# Importamos pymongo
import pymongo

Además, utilizaremos algunas librerías básicas de análisis y visualización de datos como _Pandas_, _NumPy_ y _Matplotlib_:

In [4]:
# Librerías básicas de análisis de datos.
import numpy as np
import pandas as pd
import matplotlib as mpl

In [5]:
# Versiones de las librerías usadas.
!python --version
print('PyMongo', pymongo.__version__)
print('NumPy', np.__version__)
print('Pandas', pd.__version__)
print('Matplotlib', mpl.__version__)

Python 3.7.12
PyMongo 3.12.1
NumPy 1.19.5
Pandas 1.1.5
Matplotlib 3.2.2


Esta actividad se realizó con las siguientes versiones:
*  *Python*: 3.7.10
*  *PyMongo*:  3.11.4
*  *NumPy*:  1.19.5
*  *Pandas*: 1.1.5
*  *Matplotlib*:  3.2.2


## **2. Conexión al servidor de MongoDB**
---

Para iniciar una conexión al servidor de _MongoDB_ es suficiente con especificar la IP y el puerto donde está expuesto el servicio en el constructor del cliente **`pymongo.MongoClient`**. En nuestro caso utilizamos la cadena **`localhost`**, que representa la IP de la máquina local, y el puerto  **27017** en el que corre el servicio por defecto.

In [6]:
client = pymongo.MongoClient("localhost", 27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

Si la conexión se realizó correctamente con un servicio válido y disponible de *MongoDB* debería aparecer una cadena como la siguiente:
```
MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
```

En este constructor se pueden definir más detalles como la autenticación. Para más información consulte la definición del [API del constructor](https://pymongo.readthedocs.io/en/stable/api/pymongo/mongo_client.html) y la [documentación oficial](https://docs.mongodb.com/manual/reference/connection-string/) sobre el formato de URI para la conexión de MongoDB.

## **3. Operaciones en bases de datos con _PyMongo_**
---

Antes de realizar operaciones en la base de datos, es necesario ubicar o crear  las instancias de la base de datos y de la colección a usar. Para esto, es suficiente con utilizar una sintaxis similar a la de un diccionario o como una propiedad del objeto cliente. Por ejemplo, para crear una base de datos llamada **`mlds_db`** podemos hacer lo siguiente:

In [7]:
# Declaramos la base de datos (sintaxis de diccionario).
db = client['mlds_db']

db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mlds_db')

Una expresión equivalente a esta es la siguiente:

In [8]:
# Declaramos la base de datos (sintaxis de atributo).
db = client.mlds_db

db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mlds_db')

De manera similar, se pueden crear o cargar colecciones con estos dos tipos de sintaxis sobre el objeto de base de datos. Por ejemplo, para crear una colección llamada **`mlds_col`** podemos hacer lo siguiente:

In [9]:
# Declaramos la colección (sintaxis de atributo).
col = db.mlds_col

# Expresión equivalente con sintaxis de diccionario.
#col = db['mlds_col']  

col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mlds_db'), 'mlds_col')

Para retomar el ejemplo enunciado en el taller guiado de operaciones de consola, utilizaremos la base de datos **`mlds`** y la colección **`agenda`**.

In [10]:
db = client.mlds
db.agenda

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mlds'), 'agenda')

### **3.1. Operaciones básicas (CRUD)**
---

En _PyMongo_ se dispone de funciones equivalentes a las usadas en las _shell_ de _MongoDB_ sobre el objeto colección, con algunas diferencias menores. Por ejemplo, en _PyMongo_ la función **`insert`** se distingue entre la versión para modificar un solo valor (**`insert_one`**) y para modificar múltiples valores (**`insert_many`**). 

En la _shell_ de _Mongo_ realizaríamos una inserción con una expresión como esta:

```javascript
db.agenda.insert({_id : 1,  
                  name : "Juan",  
                  age : 18,  
                  gender : "male",
                  contact : {
                           address : "Fake Street 123", phone : "555 123"
                       } 
                  })
```

Gracias a la similitud entre la sintaxis de arreglos y objetos de _JavaScript_ y la de listas y diccionarios de _Python_ la conversión de los comandos a _PyMongo_ es muy sencilla. En este caso, es suficiente con agregar comillas a los nombres de las llaves de los objetos, de la siguiente forma.

In [11]:
# Podemos acceder de la misma forma.
# '_id' en vez de _id, 'name' en vez de name, etc...
db.agenda.insert_one({'_id' : 1,       
                      'name' : "Juan", 
                      'age' : 18,  
                      'gender' : "male",
                      'contact' : {
                           'address' : "Fake Street 123", 'phone' : "555 123"
                      }})

Insertamos otro registro, tal como en la guía original:

In [12]:
db.agenda.insert_one({'_id' : 2,  
                      'name' : "Maria",  
                      'age' : 18,  
                      'gender' : "female",
                      'contact' : {
                           'address' : "Fake Street 123", 'phone' : "555 987"
                      }})

De igual forma, podemos agregar varios valores con el método **`insert_many`**, y usando la sintaxis de listas y diccionarios.

In [13]:
db.agenda.insert_many([
   {
    '_id' : 3, 'name' : "Pedro",
    'age' : 19, 'gender' : "male",
    'contact' : { 'address' : "Nowhere", 'phone' : "555 444"}
   },
   {
    '_id' : 4, 'name' : "Ana",
    'age' : 20, 'gender' : "female",
    'contact' : {'address' : "Timesquare",'phone' : "321 678"}
   }
])

Ahora, podemos obtener los valores de una consulta con la función **`find`** de la colección, tal como se haría en _Mongo_.

In [14]:
# Ejecutamos la función 'find'.
db.agenda.find()

Como puede notar, la ejecución de esta función no retorna una colección con los datos obtenidos, sino que retorna un cursor sobre el que se puede iterar. Por ejemplo, podemos obtener cada resultado individual con un bucle _for_:

In [15]:
for result in db.agenda.find():
  print(result)

{'_id': 1, 'name': 'Juan', 'age': 18, 'gender': 'male', 'contact': {'address': 'Fake Street 123', 'phone': '555 123'}}
{'_id': 2, 'name': 'Maria', 'age': 18, 'gender': 'female', 'contact': {'address': 'Fake Street 123', 'phone': '555 987'}}
{'_id': 3, 'name': 'Pedro', 'age': 19, 'gender': 'male', 'contact': {'address': 'Nowhere', 'phone': '555 444'}}
{'_id': 4, 'name': 'Ana', 'age': 20, 'gender': 'female', 'contact': {'address': 'Timesquare', 'phone': '321 678'}}


Cada uno de los resultados es un diccionario de _Python_, equivalente al objeto de _JavaScript_ que se retornaría en la _shell_. Si quisiéramos obtener solo un valor podemos utilizar el método **`find_one`**, que no retorna un cursor sino el valor directo.

In [16]:
db.agenda.find_one()

{'_id': 1,
 'age': 18,
 'contact': {'address': 'Fake Street 123', 'phone': '555 123'},
 'gender': 'male',
 'name': 'Juan'}

Para filtrar los valores con una condición, podemos realizar una expresión similar con el primer argumento del método **`find`**. Para simplificar la iteración podemos utilizar la función **`list`** de _Python_.

In [17]:
 list(db.agenda.find({'name' : "Juan"}))

[{'_id': 1,
  'age': 18,
  'contact': {'address': 'Fake Street 123', 'phone': '555 123'},
  'gender': 'male',
  'name': 'Juan'}]

Se puede utilizar expresiones especiales como **`'$in'`** y **`$gt`** envolviéndolas en una cadena de texto. Otra forma de iterar directamente los valores es usando el constructor de un _DataFrame_ de _Pandas_, que acepta listas de diccionarios para su creación.

In [18]:
pd.DataFrame(db.agenda.find({'age' : {'$in' : [18, 19]}}))

,_id,name,age,gender,contact
0,1,Juan,18,male,"{'address': 'Fake Street 123', 'phone': '555 1..."
1,2,Maria,18,female,"{'address': 'Fake Street 123', 'phone': '555 9..."
2,3,Pedro,19,male,"{'address': 'Nowhere', 'phone': '555 444'}"


A continuación, algunos de los ejemplos presentados en la guía de comandos con la _shell_ de _Mongo_.

In [19]:
# Condiciones AND.
pd.DataFrame(db.agenda.find({'age' : 18, 'gender' : "male"}))

,_id,name,age,gender,contact
0,1,Juan,18,male,"{'address': 'Fake Street 123', 'phone': '555 1..."


In [20]:
# Condiciones OR.
pd.DataFrame(db.agenda.find({
  '$or' : [
    {'age' : 18},
    {'gender' : "male"}
  ]}))

,_id,name,age,gender,contact
0,1,Juan,18,male,"{'address': 'Fake Street 123', 'phone': '555 1..."
1,2,Maria,18,female,"{'address': 'Fake Street 123', 'phone': '555 9..."
2,3,Pedro,19,male,"{'address': 'Nowhere', 'phone': '555 444'}"


In [21]:
# Condiciones compuestas de AND y OR.
pd.DataFrame(db.agenda.find({
  "contact.address" : "Fake Street 123",
  '$or' : [
    {'age' : 18},
    {'gender' : "male"}
  ]}))

,_id,name,age,gender,contact
0,1,Juan,18,male,"{'address': 'Fake Street 123', 'phone': '555 1..."
1,2,Maria,18,female,"{'address': 'Fake Street 123', 'phone': '555 9..."


In [22]:
# Condiciones con objetos/documentos.
pd.DataFrame(db.agenda.find({
  'contact' : {'address' : "Fake Street 123", 'phone' : "555 987"}
  })
)

,_id,name,age,gender,contact
0,2,Maria,18,female,"{'address': 'Fake Street 123', 'phone': '555 9..."


In [23]:
# Condiciones con 'lt' y 'gt'
pd.DataFrame(db.agenda.find({'age' : {'$gt' : 18}}))

,_id,name,age,gender,contact
0,3,Pedro,19,male,"{'address': 'Nowhere', 'phone': '555 444'}"
1,4,Ana,20,female,"{'address': 'Timesquare', 'phone': '321 678'}"


In [24]:
# Consultas con proyección.
pd.DataFrame(db.agenda.find({'age' : 18}, {'name' : 1, 'gender' : 1, '_id' : 0}))

,name,gender
0,Juan,male
1,Maria,female


Para la actualización de los registros, se disponen de funciones para uno y varios objetos. Por ejemplo, para actualizar se utilizan las funciones **`update_one`** y **`update_many`**, respectivamente. Veamos los objetos antes de modificar.

In [25]:
# Vamos a modificar el registro con nombre 'Juan'
db.agenda.find_one({'name' : "Juan"})

{'_id': 1,
 'age': 18,
 'contact': {'address': 'Fake Street 123', 'phone': '555 123'},
 'gender': 'male',
 'name': 'Juan'}

In [26]:
# Actualización de objetos con nombre Juan en el campo concact.phone.
db.agenda.update_one({'name' : "Juan"}, {'$set' : {"contact.phone" : "Lost"}})

In [27]:
# Consultamos que el objeto ha sido modificado.
db.agenda.find_one({'name' : "Juan"})

{'_id': 1,
 'age': 18,
 'contact': {'address': 'Fake Street 123', 'phone': 'Lost'},
 'gender': 'male',
 'name': 'Juan'}

Y para múltiples objetos con el método **`update_many`** basados en una condición en el primer argumento:

In [28]:
db.agenda.update_many({'age' : 18}, {'$set' : {"age" : 21}})

In [29]:
# Vemos la colección modificada.
pd.DataFrame(db.agenda.find())  

,_id,name,age,gender,contact
0,1,Juan,21,male,"{'address': 'Fake Street 123', 'phone': 'Lost'}"
1,2,Maria,21,female,"{'address': 'Fake Street 123', 'phone': '555 9..."
2,3,Pedro,19,male,"{'address': 'Nowhere', 'phone': '555 444'}"
3,4,Ana,20,female,"{'address': 'Timesquare', 'phone': '321 678'}"


Podemos eliminar elementos con los métodos para un registro **`delete_one`** y para varios registros **`delete_many`**. Note que, una vez ejecutada la celda, volver a ejecutarla o alguna de las anteriores podría generar un error o un resultado inesperado.

In [30]:
# Eliminamos un registro con edad 19.
db.agenda.delete_one({'age': 19})

In [31]:
# El objeto ya no se encuentra en la colección.
pd.DataFrame(db.agenda.find())

,_id,name,age,gender,contact
0,1,Juan,21,male,"{'address': 'Fake Street 123', 'phone': 'Lost'}"
1,2,Maria,21,female,"{'address': 'Fake Street 123', 'phone': '555 9..."
2,4,Ana,20,female,"{'address': 'Timesquare', 'phone': '321 678'}"


También podemos eliminar varios con el método **`delete_many`**:

In [32]:
db.agenda.delete_many({'age': 21})

In [33]:
# Los objetos ya no se encuentran en la colección.
pd.DataFrame(db.agenda.find())

,_id,name,age,gender,contact
0,4,Ana,20,female,"{'address': 'Timesquare', 'phone': '321 678'}"


### **3.2. Búsqueda de texto**
---
Para trabajar con búsqueda de cadenas de texto es necesaria la creación de índices, que no se realizan de la misma manera que en la _shell_. Antes de comenzar, crearemos una nueva colección y le ingresaremos los valores iniciales.


In [34]:
# Base de datos 'stores'
db = client.stores

# Colección 'stores'
db.stores.insert_many([
  {'_id': 1, 'name': "Java Hut", 'description': "Coffee and cakes"},
  {'_id': 2, 'name': "Burger Buns", 'description': "Gourmet hamburgers"},
  {'_id': 3, 'name': "Coffee Shop", 'description': "Just coffee"},
  {'_id': 4, 'name': "Clothes Clothes Clothes", 'description': "Discount clothing"},
  {'_id': 5, 'name': "Java Shopping", 'description': "Indonesian goods"}             
])

for result in db.stores.find():
  print(result)

{'_id': 1, 'name': 'Java Hut', 'description': 'Coffee and cakes'}
{'_id': 2, 'name': 'Burger Buns', 'description': 'Gourmet hamburgers'}
{'_id': 3, 'name': 'Coffee Shop', 'description': 'Just coffee'}
{'_id': 4, 'name': 'Clothes Clothes Clothes', 'description': 'Discount clothing'}
{'_id': 5, 'name': 'Java Shopping', 'description': 'Indonesian goods'}


Para crear un índice, no se ingresa como un diccionario sino como una lista de tuplas, con el primer valor siendo su nombre y el segundo su tipo. Crearemos un índice para el nombre y la descripción con el método **`create_index`**.

In [35]:
db.stores.create_index([('name', 'text'), ('description', 'text')])

'name_text_description_text'

Una vez creado el índice, las búsquedas en texto se realizan con el método **`find`**, tal como se vio en la sección anterior, envolviendo entre comillas las palabras claves. Por ejemplo, para realizar la búsqueda descrita en la guía:

In [36]:
# Búsqueda de texto básica.
for result in db.stores.find({'$text': {'$search': "java coffee shop"}}):
  print(result)

{'_id': 3, 'name': 'Coffee Shop', 'description': 'Just coffee'}
{'_id': 1, 'name': 'Java Hut', 'description': 'Coffee and cakes'}
{'_id': 5, 'name': 'Java Shopping', 'description': 'Indonesian goods'}


In [37]:
# Búsqueda de texto con cadenas exactas.
for result in db.stores.find({'$text': {'$search': "java \"coffee shop\""}}):
  print(result)

{'_id': 3, 'name': 'Coffee Shop', 'description': 'Just coffee'}


In [38]:
# Búsqueda de texto con cadenas excluidas.
for result in db.stores.find({'$text': {'$search': "java shop -coffee"}}):
  print(result)

{'_id': 5, 'name': 'Java Shopping', 'description': 'Indonesian goods'}


Para el último ejemplo, basado en el puntaje de semejanza de texto es necesario utilizar el método **`sort`** para ordenar con respecto a este puntaje. El comando original era el siguiente:

```javascript
db.stores.find(
          {$text : {$search : "java coffee shop"}},
          {score : {$meta : "textScore"}}
          ).sort({score : {$meta : "textScore"}})
```

En _PyMongo_, en vez de utilizar un diccionario como argumento, se debe indicar las variables usadas para ordenar en forma lista de tuplas, o separados con coma con el tipo de ordenamiento en la segunda posición. 

Este sería el código de _PyMongo_ equivalente al comando descrito en la guía:

In [39]:
for result in db.stores.find({'$text' : {'$search' : "java coffee shop"}},
                             {'score' : {'$meta' : "textScore"}}
                             ).sort('score', {'$meta' : "textScore"}):
  print(result)

{'_id': 3, 'name': 'Coffee Shop', 'description': 'Just coffee', 'score': 2.25}
{'_id': 1, 'name': 'Java Hut', 'description': 'Coffee and cakes', 'score': 1.5}
{'_id': 5, 'name': 'Java Shopping', 'description': 'Indonesian goods', 'score': 1.5}


### **3.3. Map-Reduce**
---

Finalmente, vamos a revisar el proceso para ejecutar operaciones _Map-Reduce_ con _Mongo_ con _PyMongo_.



In [40]:
#@markdown **Animación: Operación de agregación y _Map Reduce_ con _MongoDB_**

from IPython.display import HTML

HTML('<iframe style="width:768px; height: 432px;" src="https://drive.google.com/file/d/1no3QSSQd4vvfwY0tkeu1UHcIfIpGGqmU/preview"></iframe>')

Inicialmente, crearemos y llenaremos una colección con los datos necesarios.

In [41]:
db = client.mapreduce
db.ordenes.insert_many([{'id_cliente': 'A123', 'cantidad': 500, 'estado': 'A'},
                        {'id_cliente': 'A123', 'cantidad': 250, 'estado': 'A'},
                        {'id_cliente': 'B212', 'cantidad': 200, 'estado': 'A'},
                        {'id_cliente': 'A123', 'cantidad': 300, 'estado': 'D'}])


for result in db.ordenes.find():
  print(result)

{'_id': ObjectId('619d6c569cb51240be36ca30'), 'id_cliente': 'A123', 'cantidad': 500, 'estado': 'A'}
{'_id': ObjectId('619d6c569cb51240be36ca31'), 'id_cliente': 'A123', 'cantidad': 250, 'estado': 'A'}
{'_id': ObjectId('619d6c569cb51240be36ca32'), 'id_cliente': 'B212', 'cantidad': 200, 'estado': 'A'}
{'_id': ObjectId('619d6c569cb51240be36ca33'), 'id_cliente': 'A123', 'cantidad': 300, 'estado': 'D'}


Para realizar operaciones de agregación, es suficiente con usar la misma sintaxis que en la _shell_ de _MongoDB_, encerrando entre comillas las palabras reservadas usadas en las llaves de los diccionarios. Por ejemplo, realizando la agregación descrita en el video:

In [42]:
# Operación de agregación con PyMongo.
agg = db.ordenes.aggregate([
              {'$match': {'estado': 'A'}},
              {'$group': {'_id': '$id_cliente', 
                          'total': { '$sum': '$cantidad' }
                          }}
              ])

for result in agg:
    print(result)

{'_id': 'B212', 'total': 200}
{'_id': 'A123', 'total': 750}


En el momento de creación de este material, la ejecución de operaciones _Map-Reduce_ con _PyMongo_ requiere del uso de código fuente en el lenguaje _JavaScript_, tal como se hace en _shell_. Por ejemplo, para realizar el conteo de palabras se consideran las siguientes funciones:

* **Función _map_**: relaciona a cada **`id_cliente`** al valor numérico **`cantidad`**.

```javascript
  function map() {    
    emit(this.id_cliente, this.cantidad)
  } 
```

* **Función _reduce_**: retorna la suma de los valores por cada llave.

```javascript
function reduce(key, values) {
  return Array.sum(values)
}  
```

Para definir el código en _Python_ usaremos el método **`Code`** del módulo **`bson`**:

In [43]:
# Módulo bson (Binary JSON)
import bson

# Función MAP
mapper = bson.Code(
"""
function map() {    
    emit(this.id_cliente, this.cantidad)
  }   
""")

In [44]:
# Función REDUCE

reducer = bson.Code(
"""
function reduce(key, values) {
  return Array.sum(values)
}    
""")

Finalmente, utilizamos las funciones con el método **`map_reduce`**, que retorna la colección en la que se consolida la consulta.

In [45]:
map_reduce_result = db.ordenes.map_reduce(mapper, 
                                          reducer, 
                                          'thing_count', 
                                          query={"estado": "A"})

map_reduce_result

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mapreduce'), 'thing_count')

A partir de esta, podemos realizar operaciones como **`find`**:

In [46]:
for result in map_reduce_result.find():
  print(result)

{'_id': 'A123', 'value': 750.0}
{'_id': 'B212', 'value': 200.0}


## **Recursos adicionales**
---
A continuación, encontrará una lista de recursos para reforzar sus conocimientos en la base de datos no relacional _MongoDB_ y _PyMongo_, su _driver_ oficial para _Python_.

* [PyMongo Documentation](https://pymongo.readthedocs.io/en/stable/)
* [The MongoDB 4.4 Manual](https://docs.mongodb.com/manual/)
* [MongoDB University - MongoDB for Python Developers](https://university.mongodb.com/courses/M220P/about)
* [Udemy - Learn How Python Works with NoSql Database MongoDB: PyMongo](https://www.udemy.com/course/learn-how-python-works-with-mongodb-pymongo-in-9hrs/)


## **Créditos**
---

* **Profesor:** [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)
* **Asistentes docentes:**
  - Leonardo Avendaño Rocha
  - Alberto Nicolai Romero Martínez  

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*